52 million mothers in the United States live with their children.
Of those, over 4.5 million are in poverty.

That poverty rate of 9.0 percent is a bit lower than the overall poverty rate of 9.3 percent,[^spm] largely thanks to safety net programs that target families with children.
Yet, many of those programs fail to reach low-income mothers due to qualification rules and complex bureaucracies.
For example, [70 percent of single moms do not receive the full Child Tax Credit](https://tcf.org/content/report/what-a-child-allowance-like-canadas-would-do-for-child-poverty-in-america/?session=1).

[^spm]: This analysis uses the 2020 Current Population Survey March Supplement, and the [Supplemental Poverty Measure](https://www.census.gov/library/publications/2021/demo/p60-275.html).
The official SPM poverty rate was 9.1 percent in 2020, but the public use microdata that powers our analysis produces slightly different results.

A simpler alternative, which [other countries adopt](https://child-allowance.ubicenter.org/policies.html#other-countries) and [which we've written about in several other reports](https://www.ubicenter.org/tag/child-allowance/), is a universal child allowance: a flat amount given each month to parents for each of their children.

While child allowances chiefly reduce child poverty, they also lift mothers out of poverty.
For example, a \$200 per month child allowance would lift one million mothers out of poverty, at an annual cost of \$173 billion.[^funding]

[^funding]: This static analysis assumes no funding and no behavioral responses.
It uses the open source [OpenFisca US](https://openfisca.us) microsimulation model from [PolicyEngine](https://policyengine.org).

In [1]:
import plotly.express as px
from ubicenter import format_fig
from openfisca_us import Microsimulation

sim = Microsimulation()

In [2]:
# sim.calc("is_mother").sum()  # 52 million.
# sim.calc("spm_unit_is_in_spm_poverty", map_to="person").mean()  # 9.3%

In [3]:
from openfisca_us.api import *

from openfisca_us.model_api import *


class spm_unit_net_income(Variable):
    value_type = float
    entity = SPMUnit
    label = "Net income"
    definition_period = YEAR
    unit = USD

    formula = sum_of_variables(
        [
            "spm_unit_net_income_reported", "child_allowance"
        ]
    )


def get_mother_poverty(child_allowance_amount):
    
    class child_allowance(Variable):
        value_type = float
        entity = Person
        definition_period = YEAR

        def formula(person, period, parameters):
            return person("is_child", period) * child_allowance_amount * 12


    class add_child_allowance(Reform):
        def apply(self):
            self.update_variable(child_allowance)
            self.update_variable(spm_unit_net_income)

    reformed = Microsimulation(add_child_allowance)

    dfr = reformed.df(["is_mother", "state_code", "child_allowance", "spm_unit_is_in_spm_poverty"],
                       map_to="person")
    dfr.state_code = dfr.state_code.astype(str)
    dfr["mother_in_poverty"] = (dfr["is_mother"] * dfr["spm_unit_is_in_spm_poverty"]).astype(int)
    national = pd.Series(dict(
        child_allowance=child_allowance_amount,
        mother_poverty_count=dfr.mother_in_poverty.sum(),
        cost=dfr.child_allowance.sum(),
    ))
    # return dfr
    # print(dfr.dtypes)
    
    # print(dfr.groupby("state_code").child_allowance.sum())
    by_state = dfr.groupby("state_code").child_allowance.sum().reset_index().rename(columns={0: "cost"})
    by_state["mother_poverty_count"] = dfr.groupby("state_code").mother_in_poverty.sum().values
    # by_state = dfr[["child_allowance", "mother_in_poverty", "state_code"]].groupby("state_code").sum()
    # by_state.rename(columns={"child_allowance": "cost", "mother_in_poverty": "mother_poverty_count"}, inplace=True)
    by_state["child_allowance"] = child_allowance_amount
    return national, by_state


In [4]:
national_list = []
state_list = []

for i in np.arange(0, 510, 10):
    res = get_mother_poverty(i)
    national_list.append(res[0])
    state_list.append(res[1])

national_df = pd.DataFrame(national_list)
state_df = pd.concat(state_list).reset_index()

In [12]:
national_df["mother_poverty_cut"] = national_df.mother_poverty_count[0] - national_df.mother_poverty_count
national_df["mother_poverty_pct_cut"] = national_df.mother_poverty_count / national_df.mother_poverty_count[0]
national_df["hover"] = (
    "<b>A $" + national_df.child_allowance.astype(int).astype(str) +
    " monthly child allowance</b><br>" +
    "would lift " +
    (national_df.mother_poverty_cut / 1e6).round(1).astype(str) + 
    " million mothers (" + (national_df.mother_poverty_pct_cut * 100).astype(int).astype(str) +
    "%) out of poverty<br>" +
    " and cost $" +
    (national_df.cost / 1e9).astype(int).astype(str) +
    " billion per year"
    )
LABELS = dict(child_allowance="Monthly child allowance", mother_poverty_count="Mothers in poverty")
HOVER_TEMPLATE = "%{customdata[0]}<extra></extra>"
fig = px.line(
    national_df, x="child_allowance", y="mother_poverty_count", labels=LABELS, custom_data=["hover"],
    title="How a monthly child allowance would lift moms out of poverty")
fig.update_layout(yaxis_range=[0, 5e6], xaxis_tickformat="$0")
fig.update_traces(hovertemplate=HOVER_TEMPLATE)
format_fig(fig)

Child allowances would cut maternal poverty at similar relative rates across states.
For example, a \$200 monthly child allowance would cut it by about 15 percent in both California and Maryland.
In California, the policy would cost \$21 billion per year and lift 114,000 mothers out of poverty, while in Maryland, it would cost \$3.1 billion per year and lift 16,000 mothers out of poverty.

In [14]:
# Process data
import us

state_df["state_name"] = state_df.state_code.apply(lambda x: us.states.lookup(x).name)
current_mothers_in_poverty = state_df[state_df.child_allowance == 0][["state_code", "mother_poverty_count"]].rename(
    columns={"mother_poverty_count": "current_mothers_in_poverty"}
)
state_df2 = state_df.merge(current_mothers_in_poverty, on="state_code")
state_df2["mother_poverty_cut"] = state_df2.current_mothers_in_poverty - state_df2.mother_poverty_count
state_df2["mother_poverty_pct_cut"] = state_df2.mother_poverty_cut / state_df2.current_mothers_in_poverty
state_df2["hover"] = (
    "<b>A $" + state_df2.child_allowance.astype(int).astype(str) +
    " monthly child allowance in " + state_df2.state_name + "</b><br>" +
    "would lift " + (state_df2.mother_poverty_cut / 1e3).astype(int).astype(str) + 
    ",000 mothers (" + (state_df2.mother_poverty_pct_cut * 100).astype(int).astype(str) + 
    "%) out of poverty<br>" +
    "and cost $" +
    (state_df2.cost / 1e9).round(1).astype(str) +
    " billion per year"
)
# Graph.
LABELS = dict(child_allowance="Monthly child allowance", mother_poverty_count="Mothers in poverty")
HOVER_TEMPLATE = "%{customdata[0]}<extra></extra>"
fig = px.choropleth(
    state_df2,
    locations="state_code",
    locationmode="USA-states",
    color="mother_poverty_pct_cut",
    scope="usa",
    animation_frame="child_allowance",
    labels=LABELS,
    custom_data=["hover"],
    title="How a monthly child allowance would lift moms out of poverty by state",
    color_continuous_scale=px.colors.sequential.Blues,
    range_color=[0, state_df2.mother_poverty_pct_cut.max()]
)
fig.update_traces(hovertemplate=HOVER_TEMPLATE)
for frame in fig.frames:
    frame.data[0].hovertemplate = HOVER_TEMPLATE
fig.update_layout(coloraxis_showscale=False)
format_fig(fig)

Maternal poverty is associated with [depression and worsened child development](https://srcd.onlinelibrary.wiley.com/doi/abs/10.1111/1467-8624.00379).
Mothers spend about [1.7 hours per day](https://www.bls.gov/charts/american-time-use/activity-by-parent.htm) caring for children.
And of course, mothers suffer from the consequences of poverty like any other person does.
Child allowances can improve child outcomes, compensate parents for raising the next generation, and improve the well-being of moms everywhere.